In [2]:
version = "v2.2.033020"

# Assignment 3: Mining Vectors and Matrices (Part III)

In Part II of the assignment, we have implemented a few similarity/distance functions. In part III, we will see how can we use them to analyze our restaurant rating data.

Let's first import the necessary packages and dependencies and load the dataset prepared in Part I of this assignment.

In [3]:
import pandas as pd
import numpy as np

business_df = pd.read_csv('assets/montreal_business.csv')
business_df.set_index('business_id', inplace=True)

review_df = pd.read_csv('assets/montreal_user.csv')
rating_df = review_df.pivot_table(index=['business_id'], columns=['user_id'], values='stars')
rating_df.fillna(0, inplace=True)

missing_business_id = set(business_df.index) - set(rating_df.index)
business_df.drop(missing_business_id, inplace=True)

business_df.head()

,name,categories,stars,postal_code
business_id,,,,
-1xuC540Nycht_iWFeJ-dw,Romados,"Restaurants, Portuguese, Bakeries, Food, Chick...",4.5,H2W 1C8
-7bRnaHp7OHz8KW-THqP4w,Buffalo Bill Wings,"Restaurants, Chicken Wings",1.5,H4C 1G7
-92cC6-X87HQ1DE1UHOx3w,Junior,"Pan Asian, Restaurants, Asian Fusion, Filipino",3.5,H3J 1M8
-AgfhwHOYrsPKt-_xV_Ipg,Soupesoup,"Sandwiches, Restaurants, Cafes, Soup",4.0,H2W 1G8
-FDkvLmwaBrtVgYFqEWeWA,Café Juicy Lotus,"Vegetarian, Food, Salad, Restaurants, Organic ...",3.5,H4A 1C9


In [4]:
rating_df.head(5)

user_id,--BumyUHiO_7YsHurb9Hkw,--NIc98RMssgy0mSZL3vpA,-0zUpn_6kTWXHWzB85u7yA,-1MF2tosrw2WcCxeVNk81Q,-1wbglcr6x1qrUbqP1YAIA,-3gey5u7j0qj5SZNrOHIQg,-3s52C4zL_DHRK0ULG6qtg,-3uRUjmr1uEn-lLrhwRMCg,-4pYtFJbZYzKqmiOjGr4ag,-4tyD87vW73HG0mQlXk0uw,...,zwOiJvD09N1dNf-ts7wscw,zwsVQwyxkz-s4rvBCaPyZw,zyLIp7DRFqeIE1Grrh3Zng,zyYWUdaodH0h1jCZAvFRPg,zyaQNsI73cLOKttvGnj4bw,zyg4-MFtfPWmwucVazSjfw,zzKsgIF472IyDpJxmHDIpw,zzgb9L3NBT9V8BGw_DcVYw,zznMmC74DNBcBG62gj3guA,zzo--VpSQh8PpsGVeMC1dQ
business_id,,,,,,,,,,,,,,,,,,,,,
-1xuC540Nycht_iWFeJ-dw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-7bRnaHp7OHz8KW-THqP4w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-92cC6-X87HQ1DE1UHOx3w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-AgfhwHOYrsPKt-_xV_Ipg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-FDkvLmwaBrtVgYFqEWeWA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Suppose one of my favorite restaurants in Montréal is named *Modavie*, and I want to know how similar (in terms of customer ratings) it is compared to other restaurants. Let's see how we can do it.

In [5]:
modavie_id = business_df[business_df.name.str.contains("Modavie")].index[0]
print(modavie_id)

QeFsIMcIFIJcOrpthYL8jg


This is a sanity check that restaurant "Modavie" does exist in our dataset. 

### Exercise 3. Find similar restaurants (15 pts)
Once we have a similarity metric implemented, we can use it to find the most similar vectors. In this case, can you find out which restaurants are most similar to Modavie based on dot product? 

More specifically, can you implement the `find_max_dot_prod_restaurants` function to return the business_id of five restaurants that have the **largest** dot product with Modavie?

To do this, you need to compute the dot product between every restaurant and Modavie, and store the dot products as a seperate column named `modavie_dot_prod` on the `business_df` dataframe. Then, you should output the five rows with the largest dot product (ranked in decreasing order and excluding Modavie itself). 

**HINT 1** You may refer to Assignment 2(Part V) Jaccard Similarity to think about how to calculate the similarity scores for all restaurants.

**HINT 2** Use `np.dot` (instead of implementing your own dot product) to **greatly** speed up the execution.

In [6]:
def find_max_dot_prod_restaurants(top_n):
    #global rating_df, business_df
    modavie_vector = rating_df.loc[modavie_id]
    modavie_dot_prod = {}
    for i in range(rating_df.shape[0]):
        modavie_dot_prod[rating_df.index[i]] = np.dot(modavie_vector, rating_df.iloc[i])
    
    dot_prod_df = pd.DataFrame(modavie_dot_prod.items(), columns=['business_id', 'modavie_dot_prod'])
    dot_prod_df.set_index('business_id', inplace=True)

    business_df_cp = business_df.copy()
    business_df_cp = business_df_cp.join(dot_prod_df)
    business_df_cp = business_df_cp.sort_values(by='modavie_dot_prod', ascending=False)
    
    return business_df_cp[1:top_n+1]

We have provided the correct answers for you to verify your solution, if your code takes more than a few seconds to execute, please double check to ensure you are using `np.dot` instead of our DIY-ed `dot_prod`.

In [7]:
max_sim_restaurants = find_max_dot_prod_restaurants(10)
max_sim_restaurants.name

business_id
0W4lkclzZThpx3V65bVgig                    Schwartz's
5T6kFKFycym_GkhgOiysIw                   La Banquise
46Ld9Qc9nAx_A0jwclNZiw             Olive & Gourmando
s2I_Ni76bjJNK9yG60iD-Q        Maison Christian Faure
wl6w2VNMA4g6O9lu7kTvFA    Reuben's Deli & Steakhouse
wzugmCevnXuCMCF4upAf0w                          Kazu
cKdox2gt3L1Dbb7MpOPdWg             Au Pied de Cochon
9KmvrnyjWTr4sly0Dt770g                 Jardin Nelson
J6qWt6XIUmIGFHX5rQJA-w                      L'Avenue
2wFSVah7gU9ImNuBHwIaDg                         Belon
Name: name, dtype: object

In [8]:
answer = find_max_dot_prod_restaurants(5)
assert answer.iloc[0]['name'] == "Schwartz's"
assert answer.iloc[1]['name'] == "La Banquise"
assert answer.iloc[2]['name'] == "Olive & Gourmando"
assert answer.iloc[3]['name'] == "Maison Christian Faure"
assert answer.iloc[4]['name'] == "Reuben's Deli & Steakhouse"

### Exercise 4. (10 pts)
We can also characterize the similarity on a larger scale. In fact, we are curious to know if Modavie's good ratings are more similar to other local area restaurants or to other French restaurants.

In this exercise, please calculate the average **cosine similarity** between Modavie and (1) other restaurants in the same local area (defined by the **first 3 digits** of the postal code), (2) other French restaurants (restaurants with "French" in its `categories`)

**Note**:
1. Canada postal code uses a "AXB YCZ" pattern, where A, B, and C represent letters and X, Y, and Z represent numbers. For this exercise, we will only use the first 3 digits of a zip code, that is, "H2Y" for Modavie.
2.Again, you can see the wildness of real-world data: some food trucks are also listed as restaurants but they don't have fixed locations, let alone postal codes. For this exercise, we assume that they do not belong to any neighborhood.
3. Modavie itself should be excluded from the calculation.
4. We have prepared a NumPy-based `cosine_similarity` function for your convenience.

In [9]:
business_df.loc[modavie_id].postal_code

'H2Y 1Y6'

In [10]:
def cosine_similarity(vec_x, vec_y):
    return np.dot(vec_x, vec_y)/(np.linalg.norm(vec_x) * np.linalg.norm(vec_y))
def similarity_with_local_restaurant():
    modavie_vector = rating_df.loc[modavie_id]
    business_local = list(business_df[(~business_df['postal_code'].isna()) & (business_df['postal_code'].str.contains('H2Y')) &(~business_df['name'].str.contains("Modavie"))].index)
    rating_local_df = rating_df.ix[business_local]
    cosine_local_sum = 0

    for i in range(len(business_local)):
        cosine_local_sum += cosine_similarity(modavie_vector, rating_local_df.iloc[i])

    avg_cos_sim_local =   cosine_local_sum/len(business_local) 
    
    business_french = list(business_df[(~business_df['postal_code'].isna()) & (business_df['categories'].str.contains('French')) &(~business_df['name'].str.contains("Modavie"))].index)
    rating_french_df = rating_df.ix[business_french]

    cosine_french_sum = 0

    for i in range(len(business_french)):
        cosine_french_sum += cosine_similarity(modavie_vector, rating_french_df.iloc[i])

    avg_cos_sim_french =   cosine_french_sum/len(business_french) 
    return avg_cos_sim_local, avg_cos_sim_french

In [11]:
avg_cos_sim_local, avg_cos_sim_french = similarity_with_local_restaurant()
print(avg_cos_sim_local, avg_cos_sim_french)

0.019919428868434945 0.013618875603344313


In [12]:
avg_cos_sim_local, avg_cos_sim_french = similarity_with_local_restaurant()
assert abs(avg_cos_sim_local - 0.019919428868434952) < 1e-8, "[Exercise 4] Wrong value for avg_cos_sim_local."


In [13]:
avg_cos_sim_local, avg_cos_sim_french = similarity_with_local_restaurant()
assert abs(avg_cos_sim_french - 0.01361887560334431) < 1e-8, "[Exercise 4] Wrong value for avg_cos_sim_french."

After this exercise, you should be able to compute the similarity between many data vectors and find the most similar vectors of a given vector. This can be used as the foundation of many advanced algorithms such as k-nearest neighbor classification, information retrieval, or clustering,  which are the core of search engines or recommender systems. Computing the average similarity to selected groups of vectors (e.g., French restaurants) also provides a powerful tool to get deeper understanding of the data and to generate new features for downstream machine learning tasks. We encourage you to try these similarity metrics on your own data sets. 